In [16]:
import jdatetime
from c2cbet import config
from c2cbet.helper import spark


In [17]:
df_dest_card_blacklist = spark.read\
    .options(**config.orcl_10_props)\
    .jdbc(url=config.orcl_10_props['url'], table="tmp_c2cbet_dest")
df_dest_card_blacklist.show()

+------------------+-------------------+-----------------+
|DESTINATION_PREFIX|DESTINATION_POSTFIX|              CNT|
+------------------+-------------------+-----------------+
|            505416|               3034|224913.0000000000|
|            502229|               9779|140634.0000000000|
|            502908|               7650| 67425.0000000000|
|            589463|               2187| 58317.0000000000|
|            502229|               0821| 56875.0000000000|
|            589210|               7745| 46002.0000000000|
|            627760|               3348| 41024.0000000000|
|            606373|               0193| 32022.0000000000|
|            627760|               0605| 30957.0000000000|
|            627760|               7659| 29172.0000000000|
|            589210|               4985| 27250.0000000000|
|            589463|               7328| 26955.0000000000|
|            589463|               1460| 25042.0000000000|
|            589463|               2928| 24528.000000000

In [18]:
c2c_tnx = {"_id": {"$oid": "5c0c8cb2c849017c0f4e9fe1"},
        "_class": "com.digipay.payprecourse.model.payment.Payment",
        "amount": 10000,
        "bankTrackingCode": "834307007110",
        "creationDate": 1544326322979,
        "creditor": {
            "surname": "آریا صراف زاده"
        },
        "debtor": {
            "userId": "36b18a0b-40f6-42d1-9561-85f0c993f4b5"
        },
        "destination": {
            "uid": "87afb800-e371-40f7-ad2c-2de411004fed",
            "postfix": "1153",
            "prefix": "627412",
            "bank": {
                "code": "055",
                "name": "اقتصاد نوين",
                "cardPrefix": "627412",
                "uid": "973e0029-a3ac-425a-9658-cb50f219ad51"
            },
            "_class": "com.digipay.payprecourse.model.card.Card"
        },
        "exerciseDate": 1544326327161,
        "expirationDate": 1544412722979,
        "feeCharge": 5000,
        "initiator": "MOBILE",
        "ip": "89.221.92.42,172.16.15.25",
        "owner": {
            "userId": "36b18a0b-40f6-42d1-9561-85f0c993f4b5"
        },
        "productCode": "3",
        "reportRelay": True,
        "retryCorruptedData": 0,
        "source": {
            "uid": "748c55a0-bc64-49c6-9670-40f2578a29f6",
            "postfix": "9174",
            "prefix": "505785",
            "bank": {
                "code": "069",
                "name": "ایران زمین",
                "cardPrefix": "505785",
                "uid": "bb170434-a944-4c94-9287-0725b7ab5c44"
            },
            "_class": "com.digipay.payprecourse.model.card.Card"
        },
        "status": 8,
        "trackingCode": "5357932291544326322981",
        "type": "CARD_TRANSFER",
        "version": 1
    }
print(c2c_tnx)
dest_card_prefix = c2c_tnx["destination"]["prefix"]
dest_card_postfix = c2c_tnx["destination"]["postfix"]

{'_id': {'$oid': '5c0c8cb2c849017c0f4e9fe1'}, '_class': 'com.digipay.payprecourse.model.payment.Payment', 'amount': 10000, 'bankTrackingCode': '834307007110', 'creationDate': 1544326322979, 'creditor': {'surname': 'آریا صراف زاده'}, 'debtor': {'userId': '36b18a0b-40f6-42d1-9561-85f0c993f4b5'}, 'destination': {'uid': '87afb800-e371-40f7-ad2c-2de411004fed', 'postfix': '1153', 'prefix': '627412', 'bank': {'code': '055', 'name': 'اقتصاد نوين', 'cardPrefix': '627412', 'uid': '973e0029-a3ac-425a-9658-cb50f219ad51'}, '_class': 'com.digipay.payprecourse.model.card.Card'}, 'exerciseDate': 1544326327161, 'expirationDate': 1544412722979, 'feeCharge': 5000, 'initiator': 'MOBILE', 'ip': '89.221.92.42,172.16.15.25', 'owner': {'userId': '36b18a0b-40f6-42d1-9561-85f0c993f4b5'}, 'productCode': '3', 'reportRelay': True, 'retryCorruptedData': 0, 'source': {'uid': '748c55a0-bc64-49c6-9670-40f2578a29f6', 'postfix': '9174', 'prefix': '505785', 'bank': {'code': '069', 'name': 'ایران زمین', 'cardPrefix': '505

In [19]:
dest_card_prefix = "585983"
dest_card_postfix = "4823"

In [20]:
cnt = df_dest_card_blacklist.where("DESTINATION_PREFIX=%s and DESTINATION_POSTFIX=%s"%(dest_card_prefix,dest_card_postfix)).count()
print("IS blacklist" if cnt==1 else "NOT blacklist")


IS blacklist


In [21]:
df_c2cbet_setting = spark.read.options(**config.orcl_10_props)\
    .jdbc(url=config.orcl_10_props['url'],table="c2cbet_setting")
df_c2cbet_setting.show()

+--------------------+--------------------+
|                 KEY|               VALUE|
+--------------------+--------------------+
|bet_probability_r...|{"HIGH":{"total":...|
+--------------------+--------------------+



In [22]:
bet_probability_ranges_str = df_c2cbet_setting[df_c2cbet_setting.KEY=='bet_probability_ranges'].collect()[0]["VALUE"]
print(bet_probability_ranges_str)
# bet_probability_ranges = json.loads(bet_probability_ranges_str)
# bet_probability_ranges = yaml.load(bet_probability_ranges_str,yaml.CUnsafeLoader)
bet_probability_ranges = eval(bet_probability_ranges_str)
print(bet_probability_ranges)
print(bet_probability_ranges["HIGH"]["total"])


{"HIGH":{"total":range(2001,9999999),"month":range(1001,9999999),"day":range(51,999999)}, "MID" :{"total":range(1001,2000),"month":range(201,1000),"day":range(21,50)}, "LOW" :{"total":range(500,1000),"month":range(100,200),"day":range(10,20)},}
{'HIGH': {'total': range(2001, 9999999), 'month': range(1001, 9999999), 'day': range(51, 999999)}, 'MID': {'total': range(1001, 2000), 'month': range(201, 1000), 'day': range(21, 50)}, 'LOW': {'total': range(500, 1000), 'month': range(100, 200), 'day': range(10, 20)}}
range(2001, 9999999)


In [23]:
df_dest_card_prob_bet = spark.read.options(**config.orcl_10_props)\
    .jdbc(config.orcl_10_props["url"],table="tmp_c2cbet_prob")
df_dest_card_prob_bet.show()

+------------------+-------------------+----------------+
|DESTINATION_PREFIX|DESTINATION_POSTFIX|             CNT|
+------------------+-------------------+----------------+
|            505416|               9726|11998.0000000000|
|            606373|               6209| 9156.0000000000|
|            639599|               3326| 8994.0000000000|
|            585983|               1399| 7723.0000000000|
|            589463|               8635| 7649.0000000000|
|            589463|               0276| 7647.0000000000|
|            585983|               7356| 7600.0000000000|
|            639599|               3315| 7280.0000000000|
|            502229|               9054| 6852.0000000000|
|            585983|               3788| 6610.0000000000|
|            585983|               8602| 6474.0000000000|
|            585983|               0252| 6386.0000000000|
|            589463|               3095| 6271.0000000000|
|            606373|               2538| 6115.0000000000|
|            5

In [24]:
def is_in_probability(tnx_count):
    for _level in bet_probability_ranges.keys():
        for _time in bet_probability_ranges[_level].keys():
            if tnx_count in bet_probability_ranges[_level][_time]:
                return _level,_time

In [25]:
dest_card_prefix = "589463"
dest_card_postfix = "8635"

In [26]:
df = df_dest_card_prob_bet.where("DESTINATION_PREFIX=%s and DESTINATION_POSTFIX=%s"%(dest_card_prefix,dest_card_postfix))
if df.count()>0:
    tnx_count = df.collect()[0]["CNT"]
    _level,_time = is_in_probability(tnx_count)
    print("bet probability is:%s"%(_level if _level else None))

bet probability is:HIGH


In [27]:
df_dest_card_prob_bet_month = spark.read.options(**config.orcl_10_props) \
    .jdbc(config.orcl_10_props["url"],table="tmp_c2cbet_prob_month")
df_dest_card_prob_bet_month.show()

+------------------+-------------------+---------+---------------+
|DESTINATION_PREFIX|DESTINATION_POSTFIX|YEARMONTH|            CNT|
+------------------+-------------------+---------+---------------+
|            585983|               1399|   139911|7686.0000000000|
|            585983|               7356|   139911|7542.0000000000|
|            585983|               3788|   139909|6554.0000000000|
|            585983|               8602|   139909|6433.0000000000|
|            585983|               3784|   139911|5968.0000000000|
|            585983|               2590|   139909|5833.0000000000|
|            639599|               3326|   140008|5699.0000000000|
|            585983|               3228|   139909|5302.0000000000|
|            585983|               7997|   139909|5181.0000000000|
|            502229|               9054|   140008|5163.0000000000|
|            606373|               2538|   140008|5094.0000000000|
|            585983|               0685|   139909|5052.0000000

In [28]:
dest_card_prefix = "502229"
dest_card_postfix = "7516"

In [29]:
yearmonth= jdatetime.date.today().strftime("%Y%m")
clause = "DESTINATION_PREFIX=%s and DESTINATION_POSTFIX=%s and yearmonth=%s"%(dest_card_prefix,dest_card_postfix,yearmonth)
print(clause)
df = df_dest_card_prob_bet_month.where(clause)
if df.count()>0:
    tnx_count = df.collect()[0]["CNT"]
    print(df.count(),tnx_count)
    _level,_time = is_in_probability(tnx_count)
    print("bet probability is:%s"%(_level if _level else None))

DESTINATION_PREFIX=502229 and DESTINATION_POSTFIX=7516 and yearmonth=140101
1 359.0000000000
bet probability is:HIGH


In [30]:
df_dest_card_prob_bet_month = spark.read.options(**config.orcl_10_props)\
    .jdbc(config.orcl_10_props["url"],table="tmp_c2cbet_dest_month")
df_dest_card_prob_bet_month.show()

Py4JJavaError: An error occurred while calling o130.jdbc.
: java.sql.SQLSyntaxErrorException: ORA-00942: table or view does not exist

	at oracle.jdbc.driver.T4CTTIoer.processError(T4CTTIoer.java:445)
	at oracle.jdbc.driver.T4CTTIoer.processError(T4CTTIoer.java:396)
	at oracle.jdbc.driver.T4C8Oall.processError(T4C8Oall.java:879)
	at oracle.jdbc.driver.T4CTTIfun.receive(T4CTTIfun.java:450)
	at oracle.jdbc.driver.T4CTTIfun.doRPC(T4CTTIfun.java:192)
	at oracle.jdbc.driver.T4C8Oall.doOALL(T4C8Oall.java:531)
	at oracle.jdbc.driver.T4CPreparedStatement.doOall8(T4CPreparedStatement.java:207)
	at oracle.jdbc.driver.T4CPreparedStatement.executeForDescribe(T4CPreparedStatement.java:884)
	at oracle.jdbc.driver.OracleStatement.executeMaybeDescribe(OracleStatement.java:1167)
	at oracle.jdbc.driver.OracleStatement.doExecuteWithTimeout(OracleStatement.java:1289)
	at oracle.jdbc.driver.OraclePreparedStatement.executeInternal(OraclePreparedStatement.java:3584)
	at oracle.jdbc.driver.OracleP

In [ ]:
# for k in bet_probability_ranges.keys():
